In [4]:
import tensorflow as tf

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from keras.models import Sequential, Model

from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout,ZeroPadding2D, Add
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.input_layer import Input
from tensorflow.keras.layers import BatchNormalization
from keras.initializers import glorot_uniform

In [6]:
from keras.datasets import cifar10

(x_train,y_train), (x_test,y_test) = cifar10.load_data()

In [14]:
NUM_CLASSES = 10
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))

<Figure size 1080x288 with 0 Axes>

In [15]:
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(x_train[random_index, :])
        ax.set_title(cifar10_classes[y_train[random_index, 0]])
        plt.show()

In [9]:
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')

X_train/=255
X_test/=255

Y_train = np_utils.to_categorical(y_train, len(cifar10_classes))
Y_test = np_utils.to_categorical(y_test, len(cifar10_classes))

x_val = X_train[:10000]
partial_x_train = X_train[10000:]
y_val = Y_train[:10000]
partial_y_train = Y_train[10000:]

In [10]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                        height_shift_range=0.08, zoom_range=0.08)

val_gen = ImageDataGenerator()
train_generator = gen.flow(partial_x_train, partial_y_train, batch_size =64)
val_generator = val_gen.flow(x_val, y_val, batch_size=64)

In [11]:
def conv_block(X, f, filters, stage, block, s=2):
    conv_name = 'res'+str(stage)+block+'_branch'
    name_base = 'bn' + str(stage) + block + '_branch'
    
    # retrieve filters
    F1, F2, F3 = filters
    
    # Save the input values 
    X_shortcut = X
    
    # First component
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component
    X = Conv2D(F2, (f,f), strides=(1,1), name=conv_name + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = name_base + '2b')(X)
    X = Activation('relu')(X)
    
    # Third component
    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name + '2c', kernel_initializer=glorot_uniform(seed =0))(X)
    X = BatchNormalization(axis = 3, name = name_base + '2c')(X)
    
    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), padding = 'valid', name = conv_name + '1', kernel_initializer=glorot_uniform(seed =0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = name_base + '1')(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [12]:
X_input = Input((32,32,3))

# Stage 1
X = ZeroPadding2D(padding=(3,3), data_format=None)(X_input)
X = Conv2D(64, (7,7), strides=(2,2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides=(2,2))(X)

2022-11-13 09:40:15.784029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 09:40:15.784355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 09:40:15.784449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-13 09:40:15.784518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-13 09:40:15.784583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [13]:
X_input

<KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'input_1')>

In [31]:

X_shortcut = X


In [32]:
X.shape

TensorShape([None, 7, 7, 64])

In [33]:
# Stage 2
X = conv_block(X, f=7, filters=[64, 64, 256], stage=2, block='a', s=1)

In [34]:
# Stage 3
X = conv_block(X, f=7, filters=[64, 64, 256], stage=3, block='b', s=1)

In [35]:
# Stage 4
X = conv_block(X, f=7, filters=[64, 64, 64], stage=4, block='c', s=1)

In [36]:
X = Add()([X, X_shortcut])
X = Activation('relu')(X)

In [37]:
X = Flatten()(X)
X = Dense(NUM_CLASSES, activation='softmax', name='fc'+str(NUM_CLASSES), kernel_initializer=glorot_uniform(seed=0))(X)

In [38]:
# Create model
model = Model(inputs=X_input, outputs=X, name='ResNet50')
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 38, 38, 3)   0           ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 16, 16, 64)   9472        ['zero_padding2d_3[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1[0][0]']           

 bn4c_branch2b (BatchNormalizat  (None, 1, 1, 64)    256         ['res4c_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_21 (Activation)     (None, 1, 1, 64)     0           ['bn4c_branch2b[0][0]']          
                                                                                                  
 res4c_branch2c (Conv2D)        (None, 1, 1, 64)     4160        ['activation_21[0][0]']          
                                                                                                  
 res4c_branch1 (Conv2D)         (None, 7, 7, 64)     16448       ['activation_19[0][0]']          
                                                                                                  
 bn4c_branch2c (BatchNormalizat  (None, 1, 1, 64)    256         ['res4c_branch2c[0][0]']         
 ion)     

In [39]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=50000//64, epochs=10, validation_data=val_generator, validation_steps=10000//64)

Epoch 1/10


/tmp/ipykernel_198407/1266801284.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=50000//64, epochs=10, validation_data=val_generator, validation_steps=10000//64)


781/781 [==============================] - 29s 35ms/step - loss: 1.6861 - accuracy: 0.4200 - val_loss: 1.5179 - val_accuracy: 0.4790


In [40]:
score = model.evaluate(X_test, Y_test)


313/313 [==============================] - 2s 5ms/step - loss: 1.5174 - accuracy: 0.4760


In [41]:
score

[1.5174087285995483, 0.47600001096725464]